In [1]:
from pymongo import MongoClient
import pandas as pd
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
load_dotenv()

True

In [2]:
client = MongoClient("localhost:27017")
db = client.get_database("ironhack")
companies = db.get_collection("companies")

In [3]:
proyec = {"name":1,"category_code": 1, "_id": 0,"total_money_raised":1,  "offices.city":1, "offices.latitude":1, "offices.longitude":1}
cond = {"category_code": "games_video", "total_money_raised":{"$ne":"$0"}, "offices.latitude":{"$type": "double"},"offices.longitude":{"$type": "double"}}
list(companies.find(cond,proyec))[8]

{'name': 'Ustream',
 'category_code': 'games_video',
 'total_money_raised': '$60.1M',
 'offices': [{'city': 'San Francisco',
   'latitude': 37.392936,
   'longitude': -122.07948},
  {'city': 'Los Angeles', 'latitude': None, 'longitude': None},
  {'city': 'Budapest', 'latitude': None, 'longitude': None}]}

In [4]:
selec = pd.DataFrame(list(companies.find(cond,proyec)))

In [5]:
#lista[1]["offices"][0]["city"] #len(lista[1]["offices"])
#lista[1]["offices"][0]["latitude"] 
#lista[1]["offices"][0]["longitude"] 

In [6]:
selec.total_money_raised = selec.total_money_raised.str.strip()

In [7]:
selec["amount"] = selec.total_money_raised.str.extract(r"(.)(\d+|\d+.\d+)(.$)")[1]
selec["mil"] = selec.total_money_raised.str.extract(r"(.)(\d+|\d+.\d+)(.$)")[2]
selec["currency"] = selec.total_money_raised.str.extract(r"(.)(\d+|\d+.\d+)(.$)")[0]

In [8]:
selec.currency.unique()

array(['$', '€', '£'], dtype=object)

In [9]:
selec.mil.unique()

array(['M', 'k'], dtype=object)

In [10]:
selec.head()

,name,category_code,total_money_raised,offices,amount,mil,currency
0,Lala,games_video,$44.2M,"[{'city': 'Palo Alto', 'latitude': 37.451151, ...",44.2,M,$
1,Joost,games_video,$45M,"[{'city': 'New York', 'latitude': 40.7464969, ...",45,M,$
2,Babelgum,games_video,$13.2M,"[{'city': 'London', 'latitude': 53.344104, 'lo...",13.2,M,$
3,Kyte,games_video,$23.4M,"[{'city': 'San Francisco', 'latitude': 37.7884...",23.4,M,$
4,Veoh,games_video,$69.8M,"[{'city': 'San Diego', 'latitude': 32.902266, ...",69.8,M,$


In [11]:
dol = selec[(selec["currency"] == '$') & (selec["mil"] == "M")].sort_values("amount",ascending = False).head(1)
dol

,name,category_code,total_money_raised,offices,amount,mil,currency
85,FameCast,games_video,$9.7M,"[{'city': 'Austin', 'latitude': 30.268735, 'lo...",9.7,M,$


In [12]:
eur = selec[(selec["currency"] == '€') & (selec["mil"] == "M")].sort_values("amount",ascending = False).head(1)
eur

,name,category_code,total_money_raised,offices,amount,mil,currency
242,Bigpoint,games_video,€420M,"[{'city': 'Hamburg', 'latitude': 53.5973055, '...",420,M,€


In [13]:
pounds = selec[(selec["currency"] == '£') & (selec["mil"] == "M")].sort_values("amount",ascending = False).head(1)
pounds

,name,category_code,total_money_raised,offices,amount,mil,currency
125,amazingtunes,games_video,£1.4M,"[{'city': 'Newcastle Upon Tyne', 'latitude': 5...",1.4,M,£


In [17]:
aus = list(dol.offices.values)[0]
aus_lat = aus[0]["latitude"]
aus_lon = aus[0]["longitude"]
aus_lat, aus_lon

(30.268735, -97.745209)

In [18]:
ham = list(eur.offices.values)[0]
ham_lat = ham[0]["latitude"]
ham_lon = ham[0]["longitude"]
ham_lat, ham_lon

(53.5973055, 9.9762519)

In [19]:
ncas = list(pounds.offices.values)[0]
ncas_lat = ncas[0]["latitude"]
ncas_lon = ncas[0]["longitude"]
ncas_lat,ncas_lon

(54.971346, -1.611141)

## localizar puntos en fs

In [20]:
client_id = os.getenv("four")
client_secret = os.getenv("foursec")

In [21]:
#madrid_porsifalla = {'type': 'Point', 'coordinates': [-3.6793, 40.42955]}
Austin = {'type': 'Point', 'coordinates': [aus_lat, aus_lon]}
Hamburg = {'type': 'Point', 'coordinates': [ham_lat, ham_lon]}
Newcastle = {'type': 'Point', 'coordinates': [ncas_lat, ncas_lon]}

In [22]:
Austin, Hamburg, Newcastle

({'type': 'Point', 'coordinates': [30.268735, -97.745209]},
 {'type': 'Point', 'coordinates': [53.5973055, 9.9762519]},
 {'type': 'Point', 'coordinates': [54.971346, -1.611141]})

In [23]:
url_query = 'https://api.foursquare.com/v2/venues/search'
url_recomendados = 'https://api.foursquare.com/v2/venues/explore'

In [24]:
def parametrizacion(place,store):
    url_query = 'https://api.foursquare.com/v2/venues/search'
    parametros = {
        "client_id": client_id,
        "client_secret": client_secret,
        "v": "20180323",
        "ll": f"{place['coordinates'][0]}, {place['coordinates'][1]}",
        "query": store,
        "radius": 3000
    }
    return requests.get(url_query, params = parametros).json()

In [25]:
place = Hamburg
store = "Basketball"

Ham_bas = parametrizacion(place,store)["response"]["venues"]

In [26]:
type(Ham_bas)

list

In [27]:
Ham_bas[0]["location"]["distance"]

4150

In [28]:
place = Hamburg
store = "Conference"
ham_talks = parametrizacion(place,store)["response"]["venues"]

In [29]:
place = Hamburg
store = "school"
ham_school = parametrizacion(place,store)["response"]["venues"]

In [30]:
ham_school[0]["categories"]

[{'id': '4bf58dd8d48988d1a8941735',
  'name': 'General College & University',
  'pluralName': 'General Colleges & Universities',
  'shortName': 'Education',
  'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/education/other_',
   'suffix': '.png'},
  'primary': True}]

In [31]:
place = Hamburg
store = "design"
ham_des = parametrizacion(place,store)["response"]["venues"]

In [32]:
ham_des

[{'id': '592d4883a0215b39175ab965',
  'name': 'Schaake Design, Freelancerin für Produktdesign',
  'contact': {},
  'location': {'address': 'Ernst-Thälmann-Platz 1',
   'lat': 53.59502999999999,
   'lng': 9.984699999999975,
   'labeledLatLngs': [{'label': 'display',
     'lat': 53.59502999999999,
     'lng': 9.984699999999975}],
   'distance': 612,
   'postalCode': '20251',
   'cc': 'DE',
   'city': 'Hamburg',
   'state': 'Hamburg',
   'country': 'Deutschland',
   'formattedAddress': ['Ernst-Thälmann-Platz 1',
    '20251 Hamburg',
    'Deutschland']},
  'categories': [],
  'verified': False,
  'stats': {'tipCount': 0,
   'usersCount': 0,
   'checkinsCount': 0,
   'visitsCount': 0},
  'beenHere': {'count': 0,
   'lastCheckinExpiredAt': 0,
   'marked': False,
   'unconfirmedCount': 0},
  'hereNow': {'count': 0, 'summary': 'Nobody here', 'groups': []},
  'referralId': 'v-1636877611',
  'venueChains': [],
  'hasPerk': False},
 {'id': '51f10f34498ed64703aba8c5',
  'name': 'Christoph Burnhaus

In [33]:
place = Hamburg
store = "vegan"
ham_veg = parametrizacion(place,store)["response"]["venues"]

In [34]:
ham_veg

[{'id': '5e481aec75429600086eb591',
  'name': 'Soul - Vegan Coffee Bar',
  'contact': {},
  'location': {'address': 'Eppendorfer Landstraße 109',
   'lat': 53.593364,
   'lng': 9.988115,
   'labeledLatLngs': [{'label': 'display', 'lat': 53.593364, 'lng': 9.988115}],
   'distance': 898,
   'cc': 'DE',
   'city': 'Hamburg',
   'state': 'Hamburg',
   'country': 'Deutschland',
   'formattedAddress': ['Eppendorfer Landstraße 109',
    'Hamburg',
    'Deutschland']},
  'categories': [{'id': '4bf58dd8d48988d1d3941735',
    'name': 'Vegetarian / Vegan Restaurant',
    'pluralName': 'Vegetarian / Vegan Restaurants',
    'shortName': 'Vegetarian / Vegan',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_',
     'suffix': '.png'},
    'primary': True}],
  'verified': False,
  'stats': {'tipCount': 0,
   'usersCount': 0,
   'checkinsCount': 0,
   'visitsCount': 0},
  'beenHere': {'count': 0,
   'lastCheckinExpiredAt': 0,
   'marked': False,
   'unconfirmedCount': 0},


In [35]:
place = Hamburg
store = "airport"
ham_air = parametrizacion(place,store)["response"]["venues"]

In [36]:
ham_air[2] #categories.name = "Trail"

{'id': '5d62ec1b4d67cb0008cc2e39',
 'name': 'CarGo Autovermietung - Transporter in Hamburg Niendorf (Flughafen/Airport)',
 'contact': {},
 'location': {'address': 'Papenreye 22',
  'crossStreet': 'Kollaustrasse',
  'lat': 53.61253450631946,
  'lng': 9.963301420211792,
  'labeledLatLngs': [{'label': 'display',
    'lat': 53.61253450631946,
    'lng': 9.963301420211792}],
  'distance': 1898,
  'postalCode': '22453',
  'cc': 'DE',
  'city': 'Hamburg',
  'state': 'Hamburg',
  'country': 'Deutschland',
  'formattedAddress': ['Papenreye 22 (Kollaustrasse)',
   '22453 Hamburg',
   'Deutschland']},
 'categories': [{'id': '4bf58dd8d48988d1ef941735',
   'name': 'Rental Car Location',
   'pluralName': 'Rental Car Locations',
   'shortName': 'Rental Car',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/airport_rentalcar_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 0,
  'usersCount': 0,
  'checkinsCount': 0,
  'visitsCount': 0},
 'be

#### conseguir información de foursquare categories

In [41]:
def params_cat():
    url_cat = "https://api.foursquare.com/v2/venues/categories"
    parametros = {
        "client_id": client_id,
        "client_secret": client_secret,
        "v": "20180323"
    }
    return requests.get(url_query, params = parametros).json()
cats = params_cat()
cats

{'meta': {'code': 400,
  'errorType': 'param_error',
  'errorDetail': 'Must provide parameter ll or userll or near',
  'requestId': '6190c56c848af75a3ca817fa'},
 'response': {}}

In [42]:
url_cat = "https://api.foursquare.com/v2/venues/categories"
categ = requests.get(url_query, params = parametros).json()

NameError: name 'parametros' is not defined

In [43]:
cats["response"]["venues"]

KeyError: 'venues'

In [44]:
cats = requests.get(f"https://api.foursquare.com/v2/venues/categories&client_id={client_id}&client_secret={client_secret}").json()

In [45]:
cats

{'meta': {'code': 400,
  'errorType': 'invalid_auth',
  'errorDetail': 'Missing access credentials. See https://developer.foursquare.com/docs/api/configuration/authentication for details.',
  'requestId': '6190c5893c3d373496202f9c'},
 'response': {}}

In [46]:
import requests
from bs4 import BeautifulSoup

In [47]:
#https://developer.foursquare.com/docs/build-with-foursquare/categories/
# url = "https://developer.foursquare.com/docs/build-with-foursquare/categories/"
# html = requests.get(url)
# soup = BeautifulSoup(html.content,"html.parser")


In [48]:
# venue_cat = soup.find_all("ul", {"class": "VenueCategories__Wrapper-sc-15dn453-0 dmcDKQ"})

In [92]:
# venue_cat[0].find_all("li")[1]

<li><div><img alt="Amphitheater icon" src="https://ss3.4sqi.net/img/categories_v2/arts_entertainment/default_bg_32.png"/><h3>Amphitheater</h3><p>56aa371be4b08b9a8d5734db</p><ul class="VenueCategories__Wrapper-sc-15dn453-0 dmcDKQ"></ul></div></li>

In [80]:
# venue_cat[0].find_all("li")[940].find("h3")
# venue_cat[0].find_all("li")[940].find("p")

<p>52f2ab2ebcbc57f1066b8b4a</p>

In [81]:
li = venue_cat[0].find_all("li")

In [110]:
# #cat = []
# cat = {}
# for l in li:
#     cat[l.find("h3").getText()] = l.find("p").getText()
#     #dicc = {l.find("h3").getText():l.find("p").getText()}
#     #cat.append(dicc)

# pd.DataFrame.from_dict(cat,orient="Index").reset_index()

,index,0
0,Arts & Entertainment,4d4b7104d754a06370d81259
1,Amphitheater,56aa371be4b08b9a8d5734db
2,Aquarium,4fceea171983d5d06c3e9823
3,Arcade,4bf58dd8d48988d1e1931735
4,Art Gallery,4bf58dd8d48988d1e2931735
...,...,...
936,Train,4bf58dd8d48988d12a951735
937,Tram Station,52f2ab2ebcbc57f1066b8b51
938,Transportation Service,54541b70498ea6ccd0204bff
939,Travel Lounge,4f04b25d2fb6e1c99f3db0c0


In [117]:
def get_categories():
    url = "https://developer.foursquare.com/docs/build-with-foursquare/categories/"
    html = requests.get(url)
    soup = BeautifulSoup(html.content,"html.parser")
    venue_cat = soup.find_all("ul", {"class": "VenueCategories__Wrapper-sc-15dn453-0 dmcDKQ"})
    cat = {}
    li = venue_cat[0].find_all("li")
    for l in li:
        cat[l.find("h3").getText()] = l.find("p").getText()
        
    return pd.DataFrame.from_dict(cat,orient="Index").reset_index().rename(columns={"index": "categories", 0: "categoryId"})

categories = get_categories()

In [118]:
categories

,categories,categoryId
0,Arts & Entertainment,4d4b7104d754a06370d81259
1,Amphitheater,56aa371be4b08b9a8d5734db
2,Aquarium,4fceea171983d5d06c3e9823
3,Arcade,4bf58dd8d48988d1e1931735
4,Art Gallery,4bf58dd8d48988d1e2931735
...,...,...
936,Train,4bf58dd8d48988d12a951735
937,Tram Station,52f2ab2ebcbc57f1066b8b51
938,Transportation Service,54541b70498ea6ccd0204bff
939,Travel Lounge,4f04b25d2fb6e1c99f3db0c0


In [119]:
categories.to_csv("../data/categories.csv",index=False)